# Montandon STAC Earthquake Visualization

This notebook demonstrates how to fetch earthquake data from the Montandon STAC API and visualize it using interactive maps. We'll:

1.  Fetch the latest earthquake events, hazards, and impacts from the STAC API.
2.  Display events on an interactive global map.
3.  Allow selection of events to view related hazards and impacts on a detailed map, using ID matching logic.

In [ ]:
# Install necessary packages if not already installed
# !pip install pystac-client folium ipywidgets pandas pystac-monty

In [ ]:
import datetime
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
from pystac_client import Client

# Montandon STAC API URL
STAC_API_URL = "https://montandon-eoapi-stage.ifrc.org/stac"

In [ ]:
# Helper functions to extract Monty Extension data from STAC Items

def get_hazard_detail(item):
    """Extracts hazard detail from a STAC Item."""
    hazard_dict = item.properties.get("monty:hazard_detail")
    if not hazard_dict:
        return None
    
    # Create a simple object to hold the hazard details
    class HazardDetail:
        def __init__(self, data):
            self.cluster = data.get("cluster")
            self.severity_value = data.get("severity_value")
            self.severity_unit = data.get("severity_unit")
            self.severity_label = data.get("severity_label")
            self.estimate_type = data.get("estimate_type")
    
    return HazardDetail(hazard_dict)

def get_impact_detail(item):
    """Extracts impact detail from a STAC Item."""
    impact_dict = item.properties.get("monty:impact_detail")
    if not impact_dict:
        return None
    
    # Create a simple object to hold the impact details
    class ImpactDetail:
        def __init__(self, data):
            self.category = data.get("category")
            self.type = data.get("type")
            self.value = data.get("value")
            self.unit = data.get("unit")
            self.estimate_type = data.get("estimate_type")
    
    return ImpactDetail(impact_dict)


## 1. Connect to the STAC API and Fetch Data

We will connect to the Montandon STAC API and search for items in the `usgs-events`, `usgs-hazards`, and `usgs-impacts` collections from the last 30 days.

In [ ]:
# Define earthquake hazard codes across different classification systems
# These codes identify earthquakes in the STAC items
EARTHQUAKE_HAZARD_CODES = [
    # GLIDE classification
    "EQ",
    # EM-DAT CRED classification
    "nat-geo-ear-gro",  # Natural > Geophysical > Earthquake > Ground movement
    "nat-geo-ear-tsu",  # Natural > Geophysical > Earthquake > Tsunami
    # UNDRR-ISC 2025 Hazard Information Profiles
    "GH0101",           # Earthquake (Seismic cluster)
    # UNDRR-ISC 2020 (Historical)
    "GH0001",           # Earthquake
    "GH0002",           # Ground Shaking (Earthquake)
    "GH0003",           # Liquefaction (Earthquake Trigger)
    "GH0004",           # Earthquake Surface Rupture
    "GH0005",           # Subsidence and Uplift (Earthquake Trigger)
    "GH0006",           # Tsunami (Earthquake Trigger)
    "GH0007",           # Landslide (Earthquake Trigger)
]

# Define all event collections to search across (excluding USGS)
EVENT_COLLECTIONS = [
    "emdat-events",
    "desinventar-events",
    "gdacs-events",
    "gidd-events",
    "glide-events",
    "ifrc-events",
    "pdc-events"
]

# Define all hazard collections to search across (excluding USGS)
HAZARD_COLLECTIONS = [
    "gdacs-hazards",
    "pdc-hazards",
    "emdat-hazards",
]

# Define all impact collections to search across (excluding USGS)
IMPACT_COLLECTIONS = [
    "emdat-impacts", 
    "desinventar-impacts",
    "gdacs-impacts",
    "idmc-gidd-impacts",
    "idmc-idu-impacts",
    "ifrc-event-impacts",
    "pdc-impacts"
]


def is_earthquake(item):
    """
    Check if a STAC item is an earthquake based on its hazard codes.
    Returns True if any of the item's hazard codes match earthquake codes.
    """
    hazard_codes = item.properties.get("monty:hazard_codes", [])
    
    # Check if any hazard code matches our earthquake codes
    for code in hazard_codes:
        if code in EARTHQUAKE_HAZARD_CODES:
            return True
        # Also check for partial matches (some codes might have variations)
        code_upper = code.upper()
        if code_upper.startswith("EQ") or code_upper.startswith("GH01"):
            return True
    
    return False


def fetch_items(collection_ids, days=30, filter_earthquakes=True):
    """
    Fetch items from multiple collections for the last N days.
    
    Args:
        collection_ids: List of collection IDs to search across
        days: Number of days to look back
        filter_earthquakes: If True, only return earthquake-related items
    
    Returns:
        List of items from all collections
    """
    client = Client.open(STAC_API_URL)
    
    # Calculate the date range
    end_date = datetime.datetime.now(datetime.timezone.utc)
    start_date = end_date - datetime.timedelta(days=days)
    datetime_range = f"{start_date.isoformat()}/{end_date.isoformat()}"
    
    print(f"Searching across collections: {collection_ids}")
    print(f"Date range: {start_date} to {end_date}")
    if filter_earthquakes:
        print("Filtering for EARTHQUAKE events only")
    
    all_items = []
    collection_counts = {}
    
    # Search each collection separately and track results
    for collection_id in collection_ids:
        try:
            search = client.search(
                collections=[collection_id],
                datetime=datetime_range,
                limit=500
            )
            
            items = list(search.items())
            
            # Filter for earthquakes if requested
            if filter_earthquakes:
                earthquake_items = [item for item in items if is_earthquake(item)]
                collection_counts[collection_id] = {"total": len(items), "earthquakes": len(earthquake_items)}
                all_items.extend(earthquake_items)
                print(f"  {collection_id}: {len(earthquake_items)} earthquakes out of {len(items)} total items")
            else:
                collection_counts[collection_id] = {"total": len(items), "earthquakes": len(items)}
                all_items.extend(items)
                print(f"  Found {len(items)} items in {collection_id}")
                
        except Exception as e:
            print(f"  Error searching {collection_id}: {e}")
            collection_counts[collection_id] = {"total": 0, "earthquakes": 0}
    
    total_earthquakes = sum(c.get("earthquakes", 0) for c in collection_counts.values())
    total_items = sum(c.get("total", 0) for c in collection_counts.values())
    
    print(f"\nTotal: Found {total_earthquakes} earthquakes out of {total_items} total items")
    
    return all_items


# Fetch all EARTHQUAKE data from multiple sources
print("=" * 60)
print("FETCHING EARTHQUAKE EVENTS FROM ALL SOURCES")
print("=" * 60)
event_items = fetch_items(EVENT_COLLECTIONS, days=30, filter_earthquakes=True)

print("\n" + "=" * 60)
print("FETCHING EARTHQUAKE HAZARDS FROM ALL SOURCES")
print("=" * 60)
hazard_items = fetch_items(HAZARD_COLLECTIONS, days=30, filter_earthquakes=True)

print("\n" + "=" * 60)
print("FETCHING EARTHQUAKE IMPACTS FROM ALL SOURCES")
print("=" * 60)
# Impacts might not have hazard codes, so we'll fetch all and filter by correlation later
impact_items = fetch_items(IMPACT_COLLECTIONS, days=30, filter_earthquakes=False)

FETCHING EARTHQUAKE EVENTS FROM ALL SOURCES
Searching across collections: ['emdat-events', 'desinventar-events', 'gdacs-events', 'gidd-events', 'glide-events', 'ifrc-events', 'pdc-events']
Date range: 2025-10-26 14:11:19.650173+00:00 to 2025-11-25 14:11:19.650173+00:00
Filtering for EARTHQUAKE events only
  emdat-events: 1 earthquakes out of 6 total items
  desinventar-events: 0 earthquakes out of 0 total items
  gdacs-events: 112 earthquakes out of 130 total items
  gidd-events: 0 earthquakes out of 0 total items
  glide-events: 1 earthquakes out of 1 total items
  ifrc-events: 0 earthquakes out of 0 total items
  pdc-events: 0 earthquakes out of 1 total items

Total: Found 114 earthquakes out of 138 total items

FETCHING EARTHQUAKE HAZARDS FROM ALL SOURCES
Searching across collections: ['gdacs-hazards', 'pdc-hazards', 'emdat-hazards']
Date range: 2025-10-26 14:11:21.079317+00:00 to 2025-11-25 14:11:21.079317+00:00
Filtering for EARTHQUAKE events only
  gdacs-hazards: 25 earthquakes o

## 2. Process Events into a DataFrame

Let's convert the event items into a Pandas DataFrame for easier inspection and to prepare for visualization.

In [ ]:
def get_geometry_centroid(geometry):
    """
    Extract centroid coordinates from any geometry type.
    Returns (longitude, latitude) tuple or (None, None) if no geometry.
    """
    if not geometry:
        return None, None
    
    geom_type = geometry.get("type")
    coords = geometry.get("coordinates")
    
    if not coords:
        return None, None
    
    if geom_type == "Point":
        # Point: [lon, lat] or [lon, lat, alt]
        return coords[0], coords[1]
    
    elif geom_type == "Polygon":
        # Polygon: [[[lon, lat], [lon, lat], ...]]
        # Calculate centroid by averaging all exterior ring coordinates
        ring = coords[0]  # Exterior ring
        if ring:
            avg_lon = sum(c[0] for c in ring) / len(ring)
            avg_lat = sum(c[1] for c in ring) / len(ring)
            return avg_lon, avg_lat
    
    elif geom_type == "MultiPolygon":
        # MultiPolygon: [[[[lon, lat], ...]], [[[lon, lat], ...]]]
        all_coords = []
        for polygon in coords:
            ring = polygon[0]  # Exterior ring of each polygon
            all_coords.extend(ring)
        if all_coords:
            avg_lon = sum(c[0] for c in all_coords) / len(all_coords)
            avg_lat = sum(c[1] for c in all_coords) / len(all_coords)
            return avg_lon, avg_lat
    
    elif geom_type == "LineString":
        # LineString: [[lon, lat], [lon, lat], ...]
        if coords:
            avg_lon = sum(c[0] for c in coords) / len(coords)
            avg_lat = sum(c[1] for c in coords) / len(coords)
            return avg_lon, avg_lat
    
    elif geom_type == "MultiPoint":
        # MultiPoint: [[lon, lat], [lon, lat], ...]
        if coords:
            avg_lon = sum(c[0] for c in coords) / len(coords)
            avg_lat = sum(c[1] for c in coords) / len(coords)
            return avg_lon, avg_lat
    
    return None, None


def events_to_dataframe(events):
    """
    Convert STAC event items to a DataFrame.
    Handles events from multiple sources with different geometry types and properties.
    """
    data = []
    for item in events:
        props = item.properties
        
        # Get coordinates from any geometry type
        lon, lat = get_geometry_centroid(item.geometry)
        
        # Skip items with no valid geometry
        if lon is None or lat is None:
            print(f"Skipping item {item.id} - no valid geometry")
            continue
        
        # Get collection/source info
        source = item.collection_id if hasattr(item, 'collection_id') else "unknown"
        
        # Get hazard codes for classification
        hazard_codes = props.get("monty:hazard_codes", [])
        hazard_type = hazard_codes[0] if hazard_codes else "Unknown"
        
        # Try to get magnitude (earthquake-specific) or other severity measures
        magnitude = props.get("eq:magnitude")
        if magnitude is None:
            # Try to get from hazard_detail
            hazard_detail = props.get("monty:hazard_detail", {})
            if hazard_detail:
                magnitude = hazard_detail.get("severity_value", 0)
            else:
                magnitude = 0
        
        # Get depth if available (earthquake-specific)
        depth = props.get("eq:depth", 0)
        
        # Get country codes
        country_codes = props.get("monty:country_codes", [])
        countries = ", ".join(country_codes) if country_codes else "Unknown"
        
        data.append({
            "id": item.id,
            "title": props.get("title", "N/A"),
            "time": item.datetime,
            "source": source,
            "hazard_type": hazard_type,
            "magnitude": magnitude if magnitude else 0,
            "depth": depth if depth else 0,
            "countries": countries,
            "correlation_id": props.get("monty:correlation_id", "N/A"),
            "longitude": lon,
            "latitude": lat,
            "stac_item": item
        })
    
    df = pd.DataFrame(data)
    
    # Sort by time descending (most recent first)
    if not df.empty and "time" in df.columns:
        df = df.sort_values(by="time", ascending=False)
    
    print(f"Created DataFrame with {len(df)} events from {df['source'].nunique() if not df.empty else 0} sources")
    if not df.empty:
        print(f"Sources: {df['source'].value_counts().to_dict()}")
    
    return df

events_df = events_to_dataframe(event_items)
events_df.head(10)

Created DataFrame with 114 events from 3 sources
Sources: {'gdacs-events': 112, 'glide-events': 1, 'emdat-events': 1}


,id,title,time,source,hazard_type,magnitude,depth,countries,correlation_id,longitude,latitude,stac_item
1,gdacs-event-1511685,Earthquake in Central Mid-Atlantic Ridge,2025-11-24 02:21:02+00:00,gdacs-events,GH0101,0,0,,N/A,-37.0268,7.4819,<Item id=gdacs-event-1511685>
2,gdacs-event-1511663,Earthquake in Indonesia,2025-11-24 01:28:52+00:00,gdacs-events,GH0101,0,0,IDN,N/A,128.4260,2.4226,<Item id=gdacs-event-1511663>
3,gdacs-event-1511658,Earthquake in Chagos Archipelago Region,2025-11-24 01:22:37+00:00,gdacs-events,GH0101,0,0,,N/A,68.2746,-3.4155,<Item id=gdacs-event-1511658>
4,gdacs-event-1511657,Earthquake in Argentina,2025-11-24 01:10:56+00:00,gdacs-events,GH0101,0,0,ARG,N/A,-66.0893,-22.7929,<Item id=gdacs-event-1511657>
5,gdacs-event-1511558,Earthquake in United States,2025-11-23 08:21:46+00:00,gdacs-events,GH0101,0,0,USA,N/A,-177.0684,51.4112,<Item id=gdacs-event-1511558>
6,gdacs-event-1511556,Earthquake in Indonesia,2025-11-23 07:36:20+00:00,gdacs-events,GH0101,0,0,IDN,N/A,128.5722,1.3162,<Item id=gdacs-event-1511556>
7,gdacs-event-1511548,Earthquake in Russia,2025-11-23 05:38:21+00:00,gdacs-events,GH0101,0,0,RUS,N/A,159.1223,52.1978,<Item id=gdacs-event-1511548>
8,gdacs-event-1511543,"Earthquake in Andaman Islands, India Region",2025-11-23 04:39:22+00:00,gdacs-events,GH0101,0,0,,N/A,95.8348,13.3402,<Item id=gdacs-event-1511543>
9,gdacs-event-1511532,Earthquake in Indonesia,2025-11-23 03:19:55+00:00,gdacs-events,GH0101,0,0,IDN,N/A,128.5695,1.2749,<Item id=gdacs-event-1511532>
10,gdacs-event-1511531,Earthquake in Greenland,2025-11-23 03:03:26+00:00,gdacs-events,GH0101,0,0,GRL,N/A,-37.7653,65.7675,<Item id=gdacs-event-1511531>


## 3. Global Earthquake Map

Visualize the fetched events on a global map. Markers are sized and colored by magnitude.

In [ ]:
# Color mapping for different data sources (excluding USGS)
SOURCE_COLORS = {
    "emdat-events": "purple",
    "desinventar-events": "darkblue",
    "gdacs-events": "green",
    "gidd-events": "brown",
    "glide-events": "orange",
    "ifrc-events": "magenta",
    "pdc-events": "navy",
    "gfd-events": "teal"
}

def create_global_map(events_df):
    """
    Create a global map showing all earthquake events from multiple sources.
    Events are colored by source and sized by magnitude/severity.
    """
    if events_df.empty:
        print("No earthquake events to display")
        return folium.Map(location=[0, 0], zoom_start=2)
    
    # Filter out rows with invalid coordinates
    valid_df = events_df.dropna(subset=["latitude", "longitude"])
    
    if valid_df.empty:
        print("No events with valid coordinates")
        return folium.Map(location=[0, 0], zoom_start=2)
    
    center_lat = valid_df["latitude"].mean()
    center_lon = valid_df["longitude"].mean()
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=2, tiles="CartoDB positron")
    
    # Add title
    title_html = f"""
    <h3 align="center" style="font-size:18px; margin-top:10px;">
        <b>🌍 Earthquake Events from Multiple Sources</b>
    </h3>
    <p align="center" style="font-size:12px; color:gray;">
        Showing {len(valid_df)} earthquakes from {valid_df['source'].nunique()} data sources
    </p>
    """
    m.get_root().html.add_child(folium.Element(title_html))
    
    marker_cluster = MarkerCluster(name="Earthquakes").add_to(m)
    
    for _, row in valid_df.iterrows():
        mag = row.get("magnitude", 0) or 0
        source = row.get("source", "unknown")
        hazard_type = row.get("hazard_type", "Unknown")
        
        # Get color based on source
        color = SOURCE_COLORS.get(source, "gray")
        
        # Size based on magnitude/severity
        if mag >= 8.0:
            radius = 12
        elif mag >= 7.0:
            radius = 10
        elif mag >= 6.0:
            radius = 8
        elif mag >= 5.0:
            radius = 6
        elif mag > 0:
            radius = 5
        else:
            radius = 4  # Default size for events without magnitude
            
        # Create popup content
        popup_content = f"""
        <div style="font-size: 14px;">
            <b>{row['title']}</b><br>
            <b>Source:</b> {source}<br>
            <b>Hazard Type:</b> {hazard_type}<br>
            <b>Time:</b> {row['time']}<br>
            <b>Countries:</b> {row.get('countries', 'N/A')}<br>
            <b>Correlation ID:</b> {row.get('correlation_id', 'N/A')}<br>
        """
        
        # Add magnitude/depth only if available
        if mag > 0:
            popup_content += f"<b>Magnitude/Severity:</b> {mag}<br>"
        if row.get('depth', 0) > 0:
            popup_content += f"<b>Depth:</b> {row['depth']} km<br>"
        
        popup_content += "</div>"
        
        # Tooltip with source info
        tooltip = f"{source}: {row['title']}"
        if mag > 0:
            tooltip = f"M{mag} - {tooltip}"
        
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=radius,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=folium.Popup(popup_content, max_width=350),
            tooltip=tooltip
        ).add_to(marker_cluster)
    
    # Add a legend (excluding USGS)
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
                background-color: white; padding: 10px; border: 2px solid gray;
                border-radius: 5px; font-size: 12px;">
        <b>🌍 Earthquake Data Sources</b><br>
    """
    for source, color in SOURCE_COLORS.items():
        source_name = source.replace("-events", "").upper()
        legend_html += f'<i style="background:{color}; width:12px; height:12px; display:inline-block; border-radius:50%;"></i> {source_name}<br>'
    legend_html += "</div>"
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

global_map = create_global_map(events_df)
global_map

## 4. Selecting Events to View Hazards and Impacts

Now, let's create a simple interface to select an earthquake event and view its associated hazards and impacts on a separate map.

In [ ]:
# Define hazard and impact collections to search (excluding USGS)
HAZARD_COLLECTIONS_DETAIL = [
    "gdacs-hazards",
    "pdc-hazards",
    "emdat-hazards"
]

IMPACT_COLLECTIONS_DETAIL = [
    "emdat-impacts", 
    "desinventar-impacts",
    "gdacs-impacts",
    "idmc-gidd-impacts",
    "idmc-idu-impacts",
    "ifrc-event-impacts",
    "pdc-impacts"
]

In [ ]:
def get_correlation_id(item):
    """Extract the correlation ID from a STAC item."""
    return item.properties.get("monty:correlation_id")


def find_related_items_by_correlation(event_item):
    """
    Find related hazard and impact items using the correlation ID.
    This searches across ALL collections (excluding USGS).
    """
    correlation_id = get_correlation_id(event_item)
    
    if not correlation_id:
        print(f"No correlation ID found for event {event_item.id}")
        return [], [], []
    
    print(f"Searching for items with correlation ID: {correlation_id}")
    
    # Build CQL2 filter for correlation ID
    filter_dict = {
        "op": "=",
        "args": [
            {"property": "monty:correlation_id"},
            correlation_id
        ]
    }
    
    client = Client.open(STAC_API_URL)
    
    # Search for hazards across all hazard collections
    related_hazards = []
    hazard_sources = []
    try:
        hazard_search = client.search(
            collections=HAZARD_COLLECTIONS_DETAIL,
            filter=filter_dict,
            filter_lang="cql2-json",
            limit=100
        )
        related_hazards = list(hazard_search.items())
        
        for hazard in related_hazards:
            source = hazard.collection_id if hasattr(hazard, 'collection_id') else "unknown"
            if source not in hazard_sources:
                hazard_sources.append(source)
        
        print(f"Found {len(related_hazards)} hazards from sources: {hazard_sources}")
    except Exception as e:
        print(f"Error searching hazards: {e}")
    
    # Search for impacts across ALL impact collections
    related_impacts = []
    impact_sources = []
    try:
        impact_search = client.search(
            collections=IMPACT_COLLECTIONS_DETAIL,
            filter=filter_dict,
            filter_lang="cql2-json",
            limit=100
        )
        related_impacts = list(impact_search.items())
        
        for impact in related_impacts:
            source = impact.collection_id if hasattr(impact, 'collection_id') else "unknown"
            if source not in impact_sources:
                impact_sources.append(source)
        
        print(f"Found {len(related_impacts)} impacts from sources: {impact_sources}")
    except Exception as e:
        print(f"Error searching impacts: {e}")
    
    return related_hazards, related_impacts, impact_sources


def add_geometry_to_map(m, geometry, color, popup, tooltip, fill_opacity=0.3):
    """
    Add any geometry type to a folium map.
    Handles Point, Polygon, MultiPolygon, LineString, etc.
    """
    if not geometry:
        return
    
    geom_type = geometry.get("type")
    coords = geometry.get("coordinates")
    
    if not coords:
        return
    
    if geom_type == "Point":
        folium.CircleMarker(
            location=[coords[1], coords[0]],
            radius=8,
            color=color,
            fill=True,
            fill_opacity=0.6,
            popup=popup,
            tooltip=tooltip,
        ).add_to(m)
    
    elif geom_type == "Polygon":
        ring = coords[0]
        folium_coords = [[c[1], c[0]] for c in ring]
        folium.Polygon(
            locations=folium_coords,
            color=color,
            weight=2,
            fill=True,
            fill_opacity=fill_opacity,
            popup=popup,
            tooltip=tooltip,
        ).add_to(m)
    
    elif geom_type == "MultiPolygon":
        for polygon in coords:
            ring = polygon[0]
            folium_coords = [[c[1], c[0]] for c in ring]
            folium.Polygon(
                locations=folium_coords,
                color=color,
                weight=2,
                fill=True,
                fill_opacity=fill_opacity,
                popup=popup,
                tooltip=tooltip,
            ).add_to(m)
    
    elif geom_type == "LineString":
        folium_coords = [[c[1], c[0]] for c in coords]
        folium.PolyLine(
            locations=folium_coords,
            color=color,
            weight=3,
            popup=popup,
            tooltip=tooltip,
        ).add_to(m)


# Function to create a map showing hazards and impacts for a selected earthquake event
def create_detail_map(event_id):
    # Find the selected event
    selected_event = next((item for item in event_items if item.id == event_id), None)

    if not selected_event:
        return folium.Map(location=[0, 0], zoom_start=2)
    
    # Find related hazards and impacts using CORRELATION ID
    related_hazards, related_impacts, impact_sources = find_related_items_by_correlation(selected_event)

    # Get event coordinates using the helper function
    event_lon, event_lat = get_geometry_centroid(selected_event.geometry)
    
    if event_lon is None or event_lat is None:
        print(f"No valid geometry for event {event_id}")
        return folium.Map(location=[0, 0], zoom_start=2)

    # Create the map centered on the event
    m = folium.Map(location=[event_lat, event_lon], zoom_start=5, tiles="CartoDB positron")

    # Get event details
    props = selected_event.properties
    title = props.get("title", "Earthquake Details")
    correlation_id = get_correlation_id(selected_event) or "N/A"
    source = selected_event.collection_id if hasattr(selected_event, 'collection_id') else "unknown"
    hazard_codes = props.get("monty:hazard_codes", [])
    country_codes = props.get("monty:country_codes", [])
    
    sources_str = ", ".join(impact_sources) if impact_sources else "None found"
    
    title_html = f"""
    <h3 align="center" style="font-size:20px">
        <b>🌍 {title}</b>
    </h3>
    <p align="center" style="font-size:12px; color:gray;">
        Source: {source} | Correlation ID: {correlation_id}<br>
        Hazards: {len(related_hazards)} | Impacts: {len(related_impacts)} (from: {sources_str})
    </p>
    """
    m.get_root().html.add_child(folium.Element(title_html))

    # Create popup content for the event
    date_time = selected_event.datetime.strftime("%Y-%m-%d %H:%M:%S UTC") if selected_event.datetime else "N/A"
    
    event_popup = f"""
    <div style="font-size: 14px;">
        <b>{title}</b><br>
        <b>Source:</b> {source}<br>
        <b>Time:</b> {date_time}<br>
        <b>ID:</b> {selected_event.id}<br>
        <b>Correlation ID:</b> {correlation_id}<br>
        <b>Hazard Codes:</b> {', '.join(hazard_codes) if hazard_codes else 'N/A'}<br>
        <b>Countries:</b> {', '.join(country_codes) if country_codes else 'N/A'}<br>
    </div>
    """

    # Add event marker/geometry
    add_geometry_to_map(
        m, 
        selected_event.geometry, 
        color="red", 
        popup=folium.Popup(event_popup, max_width=300),
        tooltip="Earthquake Event Location",
        fill_opacity=0.5
    )

    # Color mapping for hazard sources (excluding USGS)
    HAZARD_COLORS = {
        "gdacs-hazards": "darkorange",
        "pdc-hazards": "coral",
        "emdat-hazards": "tomato"
    }

    # Add hazard geometries
    for hazard in related_hazards:
        hazard_detail = get_hazard_detail(hazard)
        hazard_source = hazard.collection_id if hasattr(hazard, 'collection_id') else "unknown"
        color = HAZARD_COLORS.get(hazard_source, "orange")

        hazard_popup = f"""
        <b>{hazard.properties.get("title", "Hazard")}</b><br>
        <b>Source:</b> {hazard_source}<br>
        <b>ID:</b> {hazard.id}<br>
        <b>Cluster:</b> {hazard_detail.cluster if hazard_detail else "N/A"}<br>
        <b>Severity:</b> {hazard_detail.severity_value if hazard_detail else "N/A"} 
            {hazard_detail.severity_unit if hazard_detail else ""}<br>
        <b>Estimate Type:</b> {hazard_detail.estimate_type if hazard_detail else "N/A"}<br>
        """

        add_geometry_to_map(
            m,
            hazard.geometry,
            color=color,
            popup=folium.Popup(hazard_popup, max_width=300),
            tooltip=f"Hazard ({hazard_source})",
            fill_opacity=0.2
        )

    # Color mapping for impact sources (excluding USGS)
    IMPACT_COLORS = {
        "emdat-impacts": "purple",
        "desinventar-impacts": "darkblue",
        "gdacs-impacts": "green",
        "gfd-impacts": "cyan",
        "idmc-gidd-impacts": "brown",
        "idmc-idu-impacts": "olive",
        "ifrc-event-impacts": "magenta",
        "pdc-impacts": "navy"
    }

    # Add impact geometries
    for impact in related_impacts:
        impact_detail = get_impact_detail(impact)
        impact_source = impact.collection_id if hasattr(impact, 'collection_id') else "unknown"
        color = IMPACT_COLORS.get(impact_source, "blue")
        
        # Determine label based on impact type
        if impact_detail and impact_detail.type == "imptypdeat":
            label = f"Fatalities ({impact_source})"
        elif impact_detail and impact_detail.type == "imptypcost":
            label = f"Economic Loss ({impact_source})"
        else:
            label = f"Impact ({impact_source})"

        impact_popup = f"""
        <b>{impact.properties.get("title", "Impact")}</b><br>
        <b>Source:</b> {impact_source}<br>
        <b>ID:</b> {impact.id}<br>
        <b>Category:</b> {impact_detail.category if impact_detail else "N/A"}<br>
        <b>Type:</b> {impact_detail.type if impact_detail else "N/A"}<br>
        <b>Value:</b> {impact_detail.value if impact_detail else "N/A"} {impact_detail.unit if impact_detail else ""}<br>
        <b>Estimate Type:</b> {impact_detail.estimate_type if impact_detail else "N/A"}<br>
        """

        add_geometry_to_map(
            m,
            impact.geometry,
            color=color,
            popup=folium.Popup(impact_popup, max_width=300),
            tooltip=label,
            fill_opacity=0.3
        )

    # Add layer control
    folium.LayerControl().add_to(m)
    return m

In [ ]:
# Create fresh dropdown widget for selecting earthquake events
event_options = [(f"{item.properties.get('title', 'Unknown')} ({item.collection_id})", item.id) for item in event_items]
event_dropdown = widgets.Dropdown(
    options=event_options, 
    description="Select Earthquake:", 
    style={"description_width": "initial"}, 
    layout=widgets.Layout(width="80%")
)

# Create an output widget for displaying the map
map_output = widgets.Output()

# Function to handle dropdown changes and display the map
def on_dropdown_change(change):
    # Get the selected event ID
    event_id = change['new'] if isinstance(change, dict) else change.new
    
    # Clear and recreate the output widget completely
    map_output.clear_output(wait=True)
    
    # Display inside the output widget
    with map_output:
        print(f"Loading map for: {event_id[:60]}...")
        detail_map = create_detail_map(event_id)
        # Force display with HTML to ensure refresh
        from IPython.display import HTML
        display(HTML(detail_map._repr_html_()))

# Set up the observer
event_dropdown.observe(on_dropdown_change, names='value')

# Display the widgets
display(event_dropdown)
display(map_output)

# Trigger initial map display
if event_items and event_dropdown.value:
    on_dropdown_change({'new': event_dropdown.value})

Dropdown(description='Select Earthquake:', layout=Layout(width='80%'), options=(('Ground movement in Afghanist…

Output()

In [ ]:
# Cell left empty - all widget logic is in the cell above